In [ ]:
import pymongo
cli = pymongo.MongoClient()
db = cli.test

# Approach 1: Pure linking

In [ ]:
posts = db.posts
comments = db.comments
posts.drop()
comments.drop()
comments.create_index('post_id')

In [ ]:
def make_post(title, author, text):
    result = posts.insert_one({
        'title': title,
        'author': author,
        'text': text
    })
    return result.inserted_id

In [ ]:
def make_comment(post_id, author, text):
    result = comments.insert_one({
        'post_id': post_id,
        'author': author,
        'text': text
    })
    return result.inserted_id

In [ ]:
def get_post_with_comments(post_id):
    post = posts.find_one({'_id': post_id})
    comments_ = list(comments.find({'post_id': post_id}))
    return dict(post=post, comments=comments_)

In [ ]:
post_id = make_post(
    title='First post', author='Rick', text='My very first post')
post_id

In [ ]:
comment_id = make_comment(
    post_id=post_id,
    author='Nancy',
    text='This is an uninteresting post')

In [ ]:
get_post_with_comments(post_id)

### Pure linking

*Pros*

 - Documents don't grow (more important in older MongoDB)
 - Familiar to relational users
 
*Cons*

 - Must perform 2 queries, fetching many documents, for each page view
 - Extra index comments.post_id must be created/maintained

# Pure embedding

In [ ]:
posts = db.posts
comments = db.comments
posts.drop()
comments.drop()

In [ ]:
def make_post(title, author, text):
    result = posts.insert_one({
        'title': title,
        'author': author,
        'text': text,
        'comments': []
    })
    return result.inserted_id

In [ ]:
def make_comment(post_id, author, text):
    result = posts.update_one(
        {'_id': post_id},
        {'$push': {'comments': {
            'author': author,
            'text': text}}
        })
    return post_id

In [ ]:
def get_post_with_comments(post_id):
    post = posts.find_one({'_id': post_id})
    comments = post.pop('comments')
    return dict(post=post, comments=comments)

In [ ]:
post_id = make_post(
    title='First post', author='Rick', text='My very first post')
make_comment(
    post_id=post_id,
    author='Nancy',
    text='This is an uninteresting post')

In [ ]:
get_post_with_comments(post_id)

### Pure embedding

*Pros*

 - Single query per page view
 - No extra indexes
 
*Cons*

 - Documents grow (more important in older MongoDB)
 - Unfamiliar to relational users

# Hybrid approach: pagination

In [ ]:
posts = db.posts
comment_pages = db.comment_pages
posts.drop()
comment_pages.drop()
comment_pages.create_index('post_id')

In [ ]:
def make_post(title, author, text):
    result = posts.insert_one({
        'title': title,
        'author': author,
        'text': text
    })
    return result.inserted_id

In [ ]:
def make_comment(post_id, author, text):
    result = comment_pages.update_one(
        {'post_id': post_id, 'num_comments': {'$lt': 10}},
        {'$push': {'comments': {
            'author': author,
            'text': text}},
         '$inc': {'num_comments': 1}
        },
        upsert=True)
    return result

In [ ]:
def get_post_with_comments(post_id):
    post = posts.find_one({'_id': post_id})
    q = comment_pages.find({'post_id': post_id})
    q = q.sort('_id')
    comments = []
    for pg in q:
        comments += pg['comments']
    return dict(post=post, comments=comments)

In [ ]:
post_id = make_post(
    title='First post', author='Rick', text='My very first post')
make_comment(
    post_id=post_id,
    author='Nancy',
    text='This is an uninteresting post')

In [ ]:
get_post_with_comments(post_id)

In [ ]:
for x in range(20):
    make_comment(post_id=post_id, author='spam', text='more spam')

In [ ]:
list(db.comment_pages.find({}, {'num_comments': 1}))

In [ ]:
get_post_with_comments(post_id)

### Hybrid approach with paging

 - Smaller # of documents retrieved than linking
 - Same number of indexes as linking
 - Tricky to get right
 - Posts don't grow, comment pages don't grow *much*